In [1]:
import numpy as np
import numba

In [ ]:
@numba.njit
def getDistances():
    distances = np.zeros((n, n, 2))
    for i in numba.prange(n):
        tmp = points-points[i]
        if PBC:
            tmp[tmp[:,0] >  syssize2,0] -= syssize
            tmp[tmp[:,0] < -syssize2,0] += syssize
            tmp[tmp[:,1] >  syssize2,1] -= syssize
            tmp[tmp[:,1] < -syssize2,1] += syssize
        distances[i] = tmp
    return distances

@numba.njit
def E_sys(x, y, mx, my, r_=0):
    E = np.zeros((x.size, x.size))
    #k = 1.268276778520414e-21
    k = 1
    r = getDistances()
    for i in range(x.size):
        for j in range(i+1, y.size):
            r_ij = r[i, j]
            m_i = np.array((mx[i], my[i]))
            m_j = np.array((mx[j], my[j]))
            r_ij_mod = np.sqrt((r_ij**2).sum())
            if (r_ == 0) or (r_ij_mod < r_):
                A = np.dot(m_i, m_j)/(r_ij_mod**3)
                B = np.dot(m_i, r_ij) * np.dot(m_j, r_ij) / (r_ij_mod**5)
                E[j, i] = E[i, j] = (A - 3*B) * k
    return E

PBC = True


evenFile = "unitCellEven.mfsys"
oddFile = "unitCellOdd.mfsys"
e = np.loadtxt(evenFile,skiprows=14)
o = np.loadtxt(oddFile,skiprows=14)
N = 8

head = f'''[header]
version=2
dimensions=3
type=standart
size={e.shape[0]*N*N}
emin=0.000000
emax=0.000000
state={'0'*(e.shape[0]*N*N)}
minstate=
maxstate=
interactionrange=0
sizescale=1
magnetizationscale=1
[parts]
'''

lattice = np.empty((0,8), int)

for i in range(N):
    for j in range(N):
        if (i+j)%2 == 1:
            lattice = np.vstack((lattice, e + np.array([0, 4*i, 4*j, 0, 0, 0, 0, 0])))
        else:
            lattice = np.vstack((lattice, o + np.array([0, 4*i, 4*j, 0, 0, 0, 0, 0])))
n = lattice.shape[0]

points = lattice[:, [1, 2]]
syssize = N*4
syssize2 = syssize/2

#E = E_sys(lattice[:, 1], lattice[:, 2], lattice[:, 4], lattice[:, 5])
lattice[:, 0] = np.arange(0, lattice.shape[0])
lattice[:, -1] = 0



np.savetxt(f'apamea_N{e.shape[0]*N*N}.mfsys', lattice, header=head, comments='', fmt='%d %05.5f %05.5f %05.5f %05.5f %05.5f %05.5f %d')
print(lattice.shape[0])

1280
